In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

In [ ]:
stopwords_en

In [ ]:
from string import punctuation
print('From string.punctuation:', punctuation)
# stopwords_json existe dans https://github.com/6/stopwords-json/blob/master/stopwords-all.json
stopwords_json = {"en":["a","a's","able","about","above","according","accordingly","across","actually","after","afterwards","again","against","ain't","all","allow","allows","almost","alone","along","already","also","although","always","am","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","aside","ask","asking","associated","at","available","away","awfully","b","be","became","because","become","becomes","becoming","been","before","beforehand","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","c","c'mon","c's","came","can","can't","cannot","cant","cause","causes","certain","certainly","changes","clearly","co","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","currently","d","definitely","described","despite","did","didn't","different","do","does","doesn't","doing","don't","done","down","downwards","during","e","each","edu","eg","eight","either","else","elsewhere","enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","f","far","few","fifth","first","five","followed","following","follows","for","former","formerly","forth","four","from","further","furthermore","g","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","h","had","hadn't","happens","hardly","has","hasn't","have","haven't","having","he","he's","hello","help","hence","her","here","here's","hereafter","hereby","herein","hereupon","hers","herself","hi","him","himself","his","hither","hopefully","how","howbeit","however","i","i'd","i'll","i'm","i've","ie","if","ignored","immediate","in","inasmuch","inc","indeed","indicate","indicated","indicates","inner","insofar","instead","into","inward","is","isn't","it","it'd","it'll","it's","its","itself","j","just","k","keep","keeps","kept","know","known","knows","l","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","little","look","looking","looks","ltd","m","mainly","many","may","maybe","me","mean","meanwhile","merely","might","more","moreover","most","mostly","much","must","my","myself","n","name","namely","nd","near","nearly","necessary","need","needs","neither","never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally","not","nothing","novel","now","nowhere","o","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","only","onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","own","p","particular","particularly","per","perhaps","placed","please","plus","possible","presumably","probably","provides","q","que","quite","qv","r","rather","rd","re","really","reasonably","regarding","regardless","regards","relatively","respectively","right","s","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","she","should","shouldn't","since","six","so","some","somebody","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","t","t's","take","taken","tell","tends","th","than","thank","thanks","thanx","that","that's","thats","the","their","theirs","them","themselves","then","thence","there","there's","thereafter","thereby","therefore","therein","theres","thereupon","these","they","they'd","they'll","they're","they've","think","third","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","to","together","too","took","toward","towards","tried","tries","truly","try","trying","twice","two","u","un","under","unfortunately","unless","unlikely","until","unto","up","upon","us","use","used","useful","uses","using","usually","uucp","v","value","various","very","via","viz","vs","w","want","wants","was","wasn't","way","we","we'd","we'll","we're","we've","welcome","well","went","were","weren't","what","what's","whatever","when","whence","whenever","where","where's","whereafter","whereas","whereby","wherein","whereupon","wherever","whether","which","while","whither","who","who's","whoever","whole","whom","whose","why","will","willing","wish","with","within","without","won't","wonder","would","wouldn't","x","y","yes","yet","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","z","zero"]}
stopwords_json_en = set(stopwords_json['en'])
stopwords_nltk_en = set(stopwords_en)

stopwords_punct = set(punctuation)
stoplist=set.union(stopwords_nltk_en,stopwords_punct,stopwords_json_en)

In [ ]:
def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' # if mapping isn't found, fall back to Noun.

In [ ]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

def lemmatize_sent(text): 
    # Text input is string, returns lowercased strings.
    return [lemmatizer.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in nltk.pos_tag(word_tokenize(text))]

lemmatize_sent('I was going to tell you anyway')

In [ ]:
def preprocessing(text):
    stoplist_gen=set.union(stoplist,{'http'})
    preproccessed_text=[word for word in lemmatize_sent(text) if word not in stoplist_gen]
    for word in preproccessed_text:
        if '//t.co/' in word:
            preproccessed_text.remove(word)
    return(preproccessed_text)

# Preprocessing a Data of Sentiment Analysis

In [ ]:
def preprocess_file(filename):
    tweets =[]
    tweets_cleaned = []
    labels = []
    f = open(filename)
    for line in f:
        tweet_dict = json.loads(line)
        #print(tweet_dict)
        tweets.append(tweet_dict["text"])
        tweets_cleaned.append(preprocessing(tweet_dict["text"]))
        labels.append(int(tweet_dict["label"]))
    return tweets,tweets_cleaned,labels

In [ ]:
import pandas as pd
import json
train_data = preprocess_file('../input/analyse-des-sentiment/training.json')
train_data=pd.DataFrame({"Tweets":train_data[0],"Preproccessed Tweets":train_data[1],"labels":train_data[2]})

In [ ]:
train_data

In [ ]:
test_data = preprocess_file('../input/analyse-des-sentiment/develop.json')
test_data=pd.DataFrame({"tweets":test_data[0],"Preproccessed Tweets":test_data[1],"labels":test_data[2]})

In [ ]:
test_data

# Sampling the unbalanced Train Dataset

In [ ]:
from sklearn.utils import resample

df_neg=(train_data[train_data["labels"]==-1]).sample(n=2520,random_state=42)
df_neu=(train_data[train_data["labels"]==0]).sample(n=2520,random_state=42)
df_pos=(train_data[train_data["labels"]==1]).sample(n=2520,random_state=42)

train_data=pd.concat([df_neg,df_neu,df_pos])

In [ ]:
neg=test_data[test_data.labels == 1]
print (len(neg))

In [ ]:
label_mapping={1:'Positive',-1:'Negative',0:'Neutral'}
Y=train_data['labels'].map(label_mapping)
Y

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.4)
Y.value_counts(normalize=True).plot(kind='bar', figsize=(7, 6), rot=0)
plt.xlabel("Sentiment Polarity", labelpad=14)
plt.ylabel("% of each category in the Dataset", labelpad=14)
plt.title("Distribution of tweets by polarity ", y=1.02);

# Building the word2vec model

In [ ]:
# Importing the built-in logging module
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
sentences=train_data["Preproccessed Tweets"]

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [ ]:
# Creating the model and setting values for the various parameters
num_features = 700  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size

# Initializing the train model
from gensim.models import word2vec
print("Training model....")

model = word2vec.Word2Vec(min_count=1,
                     window=10,
                     vector_size=700,
                     sample=1e-5, 
                     #alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1,
                     sg=1)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "700features_40minwords_10context"
model.save(model_name)

In [ ]:
model = word2vec.Word2Vec.load("./700features_40minwords_10context")

In [ ]:
model.build_vocab(sentences, progress_per=100000)

In [ ]:
# Nombre de tweets utilisés dans notre corpus
total_examples = model.corpus_count
print("{} tweets ont été utilisés pour créer le vocabulaire de notre corpus.".format(total_examples))

# Taille du vocabulaire de notre modèle
print("Le vocabulaire comporte {} mots distincts".format(len(model.wv.key_to_index)))

In [ ]:

model.train(sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

In [ ]:
L= "man woman girl aunt daughter".split()
L

In [ ]:
# Few tests: This will print the odd word among them 
model.wv.doesnt_match(L)

In [ ]:
La= "want guys Foreigner world".split()
La

In [ ]:
model.wv.doesnt_match(La)

In [ ]:
model.wv.most_similar("man")

In [ ]:
model.wv.most_similar("guy")

In [ ]:
import numpy as np

In [ ]:
train_tweets=train_data["Preproccessed Tweets"]
test_tweets=test_data["Preproccessed Tweets"]

In [ ]:
#Averaging Words Vectors to Create Sentence Embedding
def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in model.wv.key_to_index]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)


In [ ]:
train_array=[]
test_array=[]
for tweet in train_tweets:
    vec = get_mean_vector(model.wv, tweet)
    if len(vec) > 0:
        train_array.append(vec)
        
for tweet in test_tweets:
    vec = get_mean_vector(model.wv, tweet)
    if len(vec) > 0:
        test_array.append(vec)

In [ ]:
len(train_array[0])

# Support Vector Classifier

In [ ]:
train_tags=train_data['labels']
test_tags=test_data['labels']

In [ ]:
train_tags

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear') # SVC a linear kernel

classifier_linear.fit(train_array, train_tags)

prediction_linear = classifier_linear.predict(test_array)

Resultat=accuracy_score(test_tags,prediction_linear)

In [ ]:
Resultat

In [ ]:
## Plot confusion matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report

cf_matrix = confusion_matrix(test_tags,prediction_linear )

cf_matrix
print(classification_report(test_tags,prediction_linear))

df_cm = pd.DataFrame(cf_matrix , index = [i for i in [-1,0,1]],columns = [i for i in [-1,0,1]])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm/np.sum(df_cm), annot=True, 
            fmt='.2%',cmap="Blues")
plt.xlabel('Predicted Class')
plt.ylabel('True Class')

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=200,max_features='sqrt',random_state=0)
rf.fit(train_array, train_tags.ravel())
Y_pred=rf.predict(test_array)
rf.score(test_array,test_tags)

In [ ]:
## Plot confusion matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report

cf_matrix = confusion_matrix(test_tags,Y_pred )

cf_matrix
print(classification_report(test_tags,Y_pred))

df_cm = pd.DataFrame(cf_matrix , index = [i for i in [-1,0,1]],columns = [i for i in [-1,0,1]])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm/np.sum(df_cm), annot=True, 
            fmt='.2%',cmap="Blues")
plt.xlabel('Predicted Class')
plt.ylabel('True Class')